In [1]:
# 这是一个初始化文件，用于导入常用的库和设置一些全局的参数
import matplotlib
matplotlib.use('agg')
import matplotlib.colors as mcolors
import matplotlib.animation as ma
import matplotlib.pyplot as plt
from multiprocessing import Pool
from tqdm import tqdm
from itertools import product
import pandas as pd
import numpy as np
import numba as nb
import imageio
import sys
import os
import shutil
from main import ChiralActiveMatterNonreciprocalReact
from main import draw_mp4
from main import plot_last
import logging

randomSeed = 10

%matplotlib inline
%config InlineBackend.figure_format = "retina"

new_cmap = mcolors.LinearSegmentedColormap.from_list(
    "new", plt.cm.jet(np.linspace(0, 1, 256)) * 0.85, N=256
)

@nb.njit
def colors_idx(phaseTheta):
    return np.floor(256 - phaseTheta / (2 * np.pi) * 256).astype(np.int32)

import seaborn as sns
import matplotlib.font_manager as fm

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

param_sets = [
    {'Lambdas': [0.01], 'D0Means': [0.1], 'D0Stds': [0.1, 0.2]},
    {'Lambdas': [0.02], 'D0Means': [0.2], 'D0Stds': [0.2, 0.3, 0.4]},
    {'Lambdas': [0.02], 'D0Means': [0.4], 'D0Stds': [0.2, 0.4, 0.6, 0.8]},
    {'Lambdas': [0.4], 'D0Means': [0.3], 'D0Stds': [0.2, 0.4, 0.6]},
    {'Lambdas': [0.15], 'D0Means': [0.9], 'D0Stds': [0.6, 1.2, 1.8]},
    {'Lambdas': [0.95], 'D0Means': [2], 'D0Stds': [1, 2, 3, 4]},
]

def run_model(model):
    try:
        model.run(6000)
        # model.save()
        logging.info(f"Model with Lambda={model.strengthLambda}, D0Mean={model.distanceD0Mean}, D0Std={model.distanceD0Std} completed successfully")
    except Exception as e:
        logging.error(f"Error running model: {e}")

# def run_model(model):
#         model.run(6000)
if __name__ == "__main__":
      
    sns.set(font_scale=1.1, rc={
        'figure.figsize': (6, 5),
        'axes.facecolor': 'white',
        'figure.facecolor': 'white',
        'grid.color': '#dddddd',
        'grid.linewidth': 0.5,
        "lines.linewidth": 1.5,
        'text.color': '#000000',
        'figure.titleweight': "bold",
        'xtick.color': '#000000',
        'ytick.color': '#000000'
    })

    plt.rcParams['mathtext.fontset'] = 'stix'
    plt.rcParams['font.family'] = 'STIXGeneral'
    plt.rcParams['animation.ffmpeg_path'] = "D:/ffmpeg/bin/ffmpeg.exe"

# 单手性，d0均匀分布宽度0.1
    # Lambdas = [0.8]
    # D0Means_range = np.arange(0.05, 1.05, 0.05)
    # D0Means = np.append(D0Means_range, 2)
    # D0Means = [0.7]
    # ODs = {'uniform', 'normal', 'lorentzian'}

    models = []
    for param_set in param_sets:
        for Lambda, D0Mean, D0Std in product(param_set['Lambdas'], param_set['D0Means'], param_set['D0Stds']):
            model = ChiralActiveMatterNonreciprocalReact(
                chiralNum=1, 
                strengthLambda=Lambda, 
                distanceD0Mean=D0Mean, 
                distanceD0Std=D0Std, 
                d0Distribution='uniform', 
                omegaDistribution='uniform', 
                tqdm=True, 
                savePath="./data/", 
                overWrite=True
            )
            models.append(model)
    # models = [
    #     ChiralActiveMatterNonreciprocalReact(chiralNum = 1, 
    #                                          strengthLambda = Lambda, 
    #                                          distanceD0Mean = D0Mean, distanceD0Std=0.1, d0Distribution = 'uniform', 
    #                                          omegaDistribution = 'uniform', 
    #                                          tqdm=True, savePath="./data/", overWrite=True
    #     )
    #     for Lambda, D0Mean, in product(Lambdas, D0Means)
    # ]
    # with Pool(processes=12) as p:
    #     p.map(run_model, models)

In [ ]:
# 这里是模型的可视化函数，用于生成动画，输入的参数是模型的实例，运行完毕后会在当文件夹mp4下生成一个mp4文件
# 注意这里的模型实例必须是已经执行过计算的(即已经run过，在data文件夹中有模型的数据文件)

for model in models:
    # plot_last(model)
    # plt.clf()
    # plt.close()
    draw_mp4(model)

In [ ]:
# 这里是模型的可视化函数，用于生成最后一帧的图像，输入的参数是模型的实例，运行完毕后会在下面显示最后一帧的图像
# 注意这里的模型实例必须是已经执行过计算的(即已经run过，在data文件夹中有模型的数据文件)
# plot_last(model)

# for j in np.arange(0.01, 0.96, 0.05):
#     for k in np.arange(0.01, 0.96, 0.05):
#         for d0 in np.arange(0.01, 0.96, 0.05):

In [ ]:
# 这里是一个模型的实例, strengthLambda是相互作用的强度(耦合强度)，distanceD0是相互作用的距离，noiseRateAlpha是空间噪声强度，可以调整不同的参数多跑跑看
# model = ChiralActiveMatterNonreciprocalReact(strengthLambda=0.95, distanceD0=2, noiseRateAlpha=0, omegaTheta= 2, distribution='uniform',  tqdm=True, savePath="./data/", overWrite=True)

In [ ]:
# 这里是模型的计算执行函数，输入的参数是模型的迭代次数(演化时间)
# model.run(5000)
# for i in np.arange(0.01, 0.96, 0.05):
#     for j in {'uniform','normal','lorentzian'}:

# 
# Lambdas = [0.01, 0.02, 0.04, 0.06, 0.08, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95]
# D0Means = [np.arange(0.05, 1.01, 0.05), 2]
# ODs = {'uniform', 'normal', 'lorentzian'}

# models = [
#     ChiralActiveMatterNonreciprocalReact(chiralNum = 1, 
#                                          strengthLambda = Lambda, 
#                                          distanceD0Mean = D0Mean, distanceD0Std=0.1, d0Distribution = 'uniform', 
#                                          omegaDistribution = OD, 
#                                          noiseRateAlpha=0,
#                                          tqdm=True, savePath="./data/", overWrite=True
#     )
#     for Lambda, D0Mean, OD in product(Lambdas, D0Means, ODs)
# ]

# model.run(100)
# draw_mp4(model)